# Múltiples Herramientas (Flota Estelar)

## Objetivos de aprendizaje
- Definir varias herramientas especializadas
- Registrar herramientas en un único agente
- Observar cómo el agente decide qué herramienta usar
- Mantener respuestas concisas centradas en el flujo

## Idea clave
Un agente puede coordinar **herramientas distintas**: búsqueda, mensajería y navegación warp.
La prioridad es entender **qué herramienta se activó** y **por qué**, no generar respuestas largas.

In [1]:
# Importaciones y configuración
import os
import asyncio
from typing import Annotated
from pydantic import Field
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("✅ Entorno cargado")

✅ Entorno cargado


## Paso 1: Definir herramientas
- `buscar_registros`: consulta información en registros de la Flota
- `enviar_mensaje`: simula comunicaciones oficiales
- `calcular_warp`: cálculos de navegación warp

In [2]:
def buscar_registros(
    consulta: Annotated[str, Field(description="La consulta de búsqueda")]
) -> str:
    """Buscar información en los registros."""
    print("buscar_registros en acción...")
    return (
        f"Resultados de búsqueda para '{consulta}': se encontraron informes, "
        f"últimas actualizaciones y manuales sobre {consulta}."
    )

def enviar_mensaje(
    destinatario: Annotated[str, Field(description="Dirección del destinatario")],
    asunto: Annotated[str, Field(description="Asunto del mensaje")],
    cuerpo: Annotated[str, Field(description="Contenido del mensaje")]
) -> str:
    """Enviar un mensaje a un destinatario."""
    print("enviar_mensaje en acción...")
    return (
        f"Mensaje enviado correctamente a {destinatario} con asunto: '{asunto}'"
    )

def calcular_warp(
    operacion: Annotated[str, Field(description="Operación: tiempo, velocidad, o energia")],
    factor_warp: Annotated[float, Field(description="Factor warp (p. ej., 4.5, 6, 7)")],
    distancia_ly: Annotated[float, Field(description="Distancia en años luz")],
) -> str:
    """Calcular parámetros de navegación warp."""
    print("calcular_warp en acción...")
    velocidad = (factor_warp ** 3) * 10
    tiempo = distancia_ly / velocidad if velocidad > 0 else 0
    energia = (factor_warp ** 2) * distancia_ly * 0.8
    if operacion == "velocidad":
        return f"Velocidad warp estimada: {velocidad:.2f} unidades para warp {factor_warp}"
    if operacion == "tiempo":
        return f"Tiempo estimado: {tiempo:.3f} unidades para {distancia_ly} ly a warp {factor_warp}"
    if operacion == "energia":
        return f"Energía estimada: {energia:.2f} unidades para {distancia_ly} ly a warp {factor_warp}"
    return "Operación no válida. Usa: tiempo, velocidad, o energia."

print("✅ Herramientas definidas")

✅ Herramientas definidas


## Paso 2: Crear el agente
Instrucciones: respuestas concisas y uso de herramientas cuando corresponda.

In [3]:
cliente = OpenAIChatClient(
    base_url=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    model_id=AZURE_OPENAI_DEPLOYMENT
)

agente = ChatAgent(
    chat_client=cliente,
    name="AgenteMultiHerramienta",
    instructions=(
        "Eres un asistente versátil de la Flota Estelar. Usa herramientas para ayudar. "
        "Cuando una herramienta devuelva un resultado, repite ese resultado tal cual, "
        "sin añadir comentarios extra. Responde de forma concisa."
    ),
    tools=[buscar_registros, enviar_mensaje, calcular_warp]
)

print("✅ Agente creado")

✅ Agente creado


## Paso 3: Pruebas con prompts
Enfocadas en el flujo y con respuestas breves.

In [4]:
async def ejecutar_pruebas():
    consultas = [
        ("Picard", "Calcula el tiempo de viaje a warp 6 para 40 años luz"),
        ("T'Pral (Vulcana)", "Busca en los registros de la Flota los protocolos de primera misión"),
        ("Worf (Klingon)", "Envía un mensaje a comunicaciones@flota-estelar.org con asunto 'Informe de estado' diciendo 'La misión sigue en curso sin incidencias'"),
    ]

    for oficial, consulta in consultas:
        print(f"\n📋 Consulta de {oficial}: {consulta}")
        resultado = await agente.run(consulta)
        print(f"🖖 Respuesta: {resultado.text}")

await ejecutar_pruebas()


📋 Consulta de Picard: Calcula el tiempo de viaje a warp 6 para 40 años luz
calcular_warp en acción...
🖖 Respuesta: Tiempo estimado: 0.019 unidades para 40.0 ly a warp 6.0

📋 Consulta de T'Pral (Vulcana): Busca en los registros de la Flota los protocolos de primera misión
buscar_registros en acción...
🖖 Respuesta: Resultados de búsqueda para 'protocolos de primera misión': se encontraron informes, últimas actualizaciones y manuales sobre protocolos de primera misión.

📋 Consulta de Worf (Klingon): Envía un mensaje a comunicaciones@flota-estelar.org con asunto 'Informe de estado' diciendo 'La misión sigue en curso sin incidencias'
enviar_mensaje en acción...
🖖 Respuesta: Mensaje enviado correctamente a comunicaciones@flota-estelar.org con asunto: 'Informe de estado'


## Resumen rápido
- Varias herramientas, un solo agente
- Respuestas cortas y centradas en el resultado de la herramienta
- Ejemplos coherentes con la Flota Estelar y distintas razas